# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,262.93,99,89,2.17,GL,1731284281
1,1,puerto natales,-51.7236,-72.4875,277.56,93,63,2.72,CL,1731284128
2,2,mabaruma,8.2000,-59.7833,299.87,82,71,2.49,GY,1731284283
3,3,pandan,14.0453,124.1694,300.12,88,100,4.86,PH,1731284284
4,4,hawaiian paradise park,19.5933,-154.9731,300.88,78,75,4.63,US,1731284286


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity" ,
    color = "City"
)

# Display the map
map_plot_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
bestcity_df=city_data_df[city_data_df["Humidity"]>=30]

# Drop any rows with null values
bestcity_df["Humidity"].dropna()

# Display sample data
bestcity_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,262.93,99,89,2.17,GL,1731284281
1,1,puerto natales,-51.7236,-72.4875,277.56,93,63,2.72,CL,1731284128
2,2,mabaruma,8.2000,-59.7833,299.87,82,71,2.49,GY,1731284283
3,3,pandan,14.0453,124.1694,300.12,88,100,4.86,PH,1731284284
4,4,hawaiian paradise park,19.5933,-154.9731,300.88,78,75,4.63,US,1731284286
...,...,...,...,...,...,...,...,...,...,...
569,569,isangel,-19.5500,169.2667,302.21,65,75,5.14,VU,1731284993
570,570,sal rei,16.1833,-22.9167,298.94,77,88,7.23,CV,1731284994
571,571,eirunepe,-6.6603,-69.8736,300.69,67,97,1.52,BR,1731284995
572,572,shimoda,34.6667,138.9500,291.41,85,100,3.64,JP,1731284996


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
bestcity_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
bestcity_df["Hotel Name"] = " "

# Display sample data
bestcity_df

/var/folders/28/rtv_kxr161z67vz7r8160lc00000gn/T/ipykernel_67604/4264704448.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bestcity_df["Hotel Name"] = " "


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,aasiaat,68.7098,-52.8699,262.93,99,89,2.17,GL,1731284281,
1,1,puerto natales,-51.7236,-72.4875,277.56,93,63,2.72,CL,1731284128,
2,2,mabaruma,8.2000,-59.7833,299.87,82,71,2.49,GY,1731284283,
3,3,pandan,14.0453,124.1694,300.12,88,100,4.86,PH,1731284284,
4,4,hawaiian paradise park,19.5933,-154.9731,300.88,78,75,4.63,US,1731284286,
...,...,...,...,...,...,...,...,...,...,...,...
569,569,isangel,-19.5500,169.2667,302.21,65,75,5.14,VU,1731284993,
570,570,sal rei,16.1833,-22.9167,298.94,77,88,7.23,CV,1731284994,
571,571,eirunepe,-6.6603,-69.8736,300.69,67,97,1.52,BR,1731284995,
572,572,shimoda,34.6667,138.9500,291.41,85,100,3.64,JP,1731284996,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
hotel_df=bestcity_df
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,aasiaat,68.7098,-52.8699,262.93,99,89,2.17,GL,1731284281,
1,1,puerto natales,-51.7236,-72.4875,277.56,93,63,2.72,CL,1731284128,
2,2,mabaruma,8.2000,-59.7833,299.87,82,71,2.49,GY,1731284283,
3,3,pandan,14.0453,124.1694,300.12,88,100,4.86,PH,1731284284,
4,4,hawaiian paradise park,19.5933,-154.9731,300.88,78,75,4.63,US,1731284286,
...,...,...,...,...,...,...,...,...,...,...,...
569,569,isangel,-19.5500,169.2667,302.21,65,75,5.14,VU,1731284993,
570,570,sal rei,16.1833,-22.9167,298.94,77,88,7.23,CV,1731284994,
571,571,eirunepe,-6.6603,-69.8736,300.69,67,97,1.52,BR,1731284995,
572,572,shimoda,34.6667,138.9500,291.41,85,100,3.64,JP,1731284996,


In [46]:
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lng = hotel_df.loc[index, "Lat"]
    Lat = hotel_df.loc[index, "Lng"]
    break
Lat

-52.8699

In [52]:
# Set parameters to search for a hotel
radius = 10000


params = {
    "categories" :"accommodation.hotel",
    "apiKey":geoapify_key ,
    "limit" :2
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
aasiaat - nearest hotel: SØMA
puerto natales - nearest hotel: Dorotea Patagonia Hostel
mabaruma - nearest hotel: Hotel Broost
pandan - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
bronnitsy - nearest hotel: Маруся
port-aux-francais - nearest hotel: Keravel
weno - nearest hotel: High Tide Hotel
iqaluit - nearest hotel: Frobisher Inn
severo-yeniseyskiy - nearest hotel: Актолик
saint croix - nearest hotel: The Buccaneer
avarua - nearest hotel: Paradise Inn
vozhega - nearest hotel: No hotel found
bubaque - nearest hotel: Chez Julio
tarawa - nearest hotel: Betio Lodge
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
tangermunde - nearest hotel: Dependance Königin Luise
anadyr - nearest hotel: Гостевой дом
lorengau - nearest hotel: Seeadler Bay Hotel
enewetak - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
grytviken - nearest hotel: No hotel found
f

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,aasiaat,68.7098,-52.8699,262.93,99,89,2.17,GL,1731284281,SØMA
1,1,puerto natales,-51.7236,-72.4875,277.56,93,63,2.72,CL,1731284128,Dorotea Patagonia Hostel
2,2,mabaruma,8.2000,-59.7833,299.87,82,71,2.49,GY,1731284283,Hotel Broost
3,3,pandan,14.0453,124.1694,300.12,88,100,4.86,PH,1731284284,No hotel found
4,4,hawaiian paradise park,19.5933,-154.9731,300.88,78,75,4.63,US,1731284286,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
569,569,isangel,-19.5500,169.2667,302.21,65,75,5.14,VU,1731284993,Tanna Lodge
570,570,sal rei,16.1833,-22.9167,298.94,77,88,7.23,CV,1731284994,Hotel Marine Club
571,571,eirunepe,-6.6603,-69.8736,300.69,67,97,1.52,BR,1731284995,No hotel found
572,572,shimoda,34.6667,138.9500,291.41,85,100,3.64,JP,1731284996,ホテルウラガ


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [59]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    hover_cols='Hotel Name',
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City"
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)